In [0]:
%run "/Workspace/Project2/MyUtilityFunctions"

[SecretScope(name='adlsstgconnection')]

In [0]:
# Initialize list to store DataFrames
# Load Silver Layer Parquet data for all files

all_data_df=[]

for i in range(0,len(files)):
    all_data_df.append(get_adls_data("/mnt/project2-container/Silver_Layer/", "parquet", files[i]\
        .replace(".csv", "")))

print(len(all_data_df))

5


In [0]:
%run "/Workspace/Project2/GoldDeltaTableCreation"

In [0]:
# Generate Hashkey column for SCD (Slowly Changing Dimensions) tracking

from pyspark.sql.functions import crc32, concat

for i in range(0,len(files)):
    all_data_df[i] = all_data_df[i].withColumn("Hashkey", crc32(concat(*all_data_df[i].columns)))


In [0]:
# Read existing Gold Layer Delta Tables

from delta.tables import DeltaTable

delta_tabels = []
delta_table_dfs = []

for i in range(0,len(files)):
    fn = files[i].replace(".csv", "")
    delta_tabels.append(DeltaTable.forPath(spark, f"/mnt/project2-container/Gold_Layer/{fn}/"))
    delta_table_dfs.append(delta_tabels[i].toDF())



In [0]:
# Define update and insert values for each table

from pyspark.sql.functions import col, lit, current_timestamp

values={
    "accounts.csv": 
        {
        "set_values":
            {
                "tgt.AccountId": "src.AccountId",
                "tgt.CustomerId":"src.CustomerId" ,
                "tgt.AccountType": "src.AccountType",
                "tgt.Balance": "src.Balance",
                "tgt.UpdatedBy": lit("Gold-Databricks-Update"),
                "tgt.UpdatedDate": current_timestamp(),
                "tgt.Hashkey": "src.Hashkey" 
            },
        "insert_values" :
            {
                "tgt.AccountId": "src.AccountId",
                "tgt.CustomerId": "src.CustomerId" ,
                "tgt.AccountType": "src.AccountType",
                "tgt.Balance": "src.Balance",
                "tgt.CreatedBy": lit("Gold-Databricks"),
                "tgt.CreatedDate": current_timestamp(),
                "tgt.UpdatedBy": lit("Gold-Databricks"),
                "tgt.UpdatedDate": current_timestamp(),
                "tgt.Hashkey": "src.Hashkey"
            }
    },
        
    "customers.csv": 
        {
        "set_values":
            {
                "tgt.CustomerId": "src.CustomerId",
                "tgt.FirstName": "src.FirstName",
                "tgt.LastName": "src.LastName",
                "tgt.Address": "src.Address",
                "tgt.City": "src.City",
                "tgt.State": "src.State",
                "tgt.Zip": "src.Zip",
                "tgt.UpdatedBy": lit("Gold-Databricks-Update"),
                "tgt.UpdatedDate": current_timestamp(),
                "tgt.Hashkey": "src.Hashkey"
            },
        "insert_values" :
            {
                "tgt.CustomerId": "src.CustomerId",
                "tgt.FirstName": "src.FirstName",
                "tgt.LastName": "src.LastName",
                "tgt.Address": "src.Address",
                "tgt.City": "src.City",
                "tgt.State": "src.State",
                "tgt.Zip": "src.Zip",
                "tgt.CreatedBy": lit("Gold-Databricks"),
                "tgt.CreatedDate": current_timestamp(),
                "tgt.UpdatedBy": lit("Gold-Databricks"),
                "tgt.UpdatedDate": current_timestamp(),
                "tgt.Hashkey": "src.Hashkey"
            },
    },
               
    "loan_payments.csv": 
    {
        "set_values": {
            "tgt.PaymentId": "src.PaymentId",
            "tgt.LoanId": "src.LoanId",
            "tgt.PaymentDate": "src.PaymentDate",
            "tgt.PaymentAmount": "src.PaymentAmount",
            "tgt.UpdatedBy": lit("Gold-Databricks-Update"),
            "tgt.UpdatedDate": current_timestamp(),
            "tgt.Hashkey": "src.Hashkey"
        },
        "insert_values": {
            "tgt.PaymentId": "src.PaymentId",
            "tgt.LoanId": "src.LoanId",
            "tgt.PaymentDate": "src.PaymentDate",
            "tgt.PaymentAmount": "src.PaymentAmount",
            "tgt.CreatedBy": lit("Gold-Databricks"),
            "tgt.CreatedDate": current_timestamp(),
            "tgt.UpdatedBy": lit("Gold-Databricks"),
            "tgt.UpdatedDate": current_timestamp(),
            "tgt.Hashkey": "src.Hashkey"
        }
    },

    "loans.csv": 
        {
        "set_values": {
            "tgt.LoanId": "src.LoanId",
            "tgt.CustomerId": "src.CustomerId",
            "tgt.LoanAmount": "src.LoanAmount",
            "tgt.InterestRate": "src.InterestRate",
            "tgt.LoanTerm": "src.LoanTerm",
            "tgt.UpdatedBy": lit("Gold-Databricks-Update"),
            "tgt.UpdatedDate": current_timestamp(),
            "tgt.Hashkey": "src.Hashkey"
        },
        "insert_values": {
            "tgt.LoanId": "src.LoanId",
            "tgt.CustomerId": "src.CustomerId",
            "tgt.LoanAmount": "src.LoanAmount",
            "tgt.InterestRate": "src.InterestRate",
            "tgt.LoanTerm": "src.LoanTerm",
            "tgt.CreatedBy": lit("Gold-Databricks"),
            "tgt.CreatedDate": current_timestamp(),
            "tgt.UpdatedBy": lit("Gold-Databricks"),
            "tgt.UpdatedDate": current_timestamp(),
            "tgt.Hashkey": "src.Hashkey"
        }
    },

    "transactions.csv": 
    {
        "set_values": {
            "tgt.TransactionId": "src.TransactionId",
            "tgt.AccountId": "src.AccountId",
            "tgt.TransactionDate": "src.TransactionDate",
            "tgt.TransactionAmount": "src.TransactionAmount",
            "tgt.TransactionType": "src.TransactionType",
            "tgt.UpdatedBy": lit("Gold-Databricks-Update"),
            "tgt.UpdatedDate": current_timestamp(),
            "tgt.Hashkey": "src.Hashkey"
        },
        "insert_values": {
            "tgt.TransactionId": "src.TransactionId",
            "tgt.AccountId": "src.AccountId",
            "tgt.TransactionDate": "src.TransactionDate",
            "tgt.TransactionAmount": "src.TransactionAmount",
            "tgt.TransactionType": "src.TransactionType",
            "tgt.CreatedBy": lit("Gold-Databricks"),
            "tgt.CreatedDate": current_timestamp(),
            "tgt.UpdatedBy": lit("Gold-Databricks"),
            "tgt.UpdatedDate": current_timestamp(),
            "tgt.Hashkey": "src.Hashkey"
        }
    }   
    
}

In [0]:
# Extract primary key mapping for each table

tgt_ids = []
src_ids = []

for i in range(0,len(files)):
    tgt_ids.append(next(iter(values[files[i]]["set_values"])))
    src_ids.append(values[files[i]]["set_values"][tgt_ids[i]])
    
print(tgt_ids)
print(src_ids)

['tgt.AccountId', 'tgt.CustomerId', 'tgt.PaymentId', 'tgt.LoanId', 'tgt.TransactionId']
['src.AccountId', 'src.CustomerId', 'src.PaymentId', 'src.LoanId', 'src.TransactionId']


In [0]:
# Identify new or changed records using anti join on Hashkey + Primary Key

src1_dfs = []

for i in range(0,len(files)):
    src1_dfs.append(all_data_df[i].alias("src").join(
        delta_table_dfs[i].alias("tgt"),
        ( 
            (col(src_ids[i]) == col(tgt_ids[i])) & (col("src.Hashkey") == col("tgt.Hashkey"))
        ),
        "anti"
    ).select("src.*"))


In [0]:
# Perform MERGE to update or insert records into Gold Layer Delta tables

for i in range(0,len(files)):
    delta_tabels[i].alias("tgt").merge(src1_dfs[i].alias("src"),((col(tgt_ids[i]) == col(src_ids[i]))))\
        .whenMatchedUpdate(
            set = values[files[i]]["set_values"]
            )\
            .whenNotMatchedInsert(
                values =  values[files[i]]["insert_values"]
            ).execute()

In [0]:
%sql

select * from accounts

AccountId,CustomerId,AccountType,Balance,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,Hashkey
54,42,Checking,5500.50,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,221025946
88,1,Checking,8900.00,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,512030229
82,2,Checking,8300.50,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,3247491694
92,44,Checking,9300.00,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,485707070
21,53,Savings,300.25,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,2800704470
29,58,Savings,75.25,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,3637748452
35,62,Savings,175.75,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,3427479153
38,15,Checking,3900.50,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,1689886302
72,17,Checking,7300.00,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,1425050024
33,85,Savings,150.25,Gold-Databricks,2025-04-24T19:57:13.858Z,Gold-Databricks,2025-04-24T19:57:13.858Z,655204919


[SecretMetadata(key='appid'),
 SecretMetadata(key='appsecret'),
 SecretMetadata(key='azure-sqldb-password'),
 SecretMetadata(key='azure-sqldb-username'),
 SecretMetadata(key='onprem-system-password')]

/mnt/project2-container is already mounted.


[FileInfo(path='dbfs:/mnt/project2-container/Bronze_Layer/', name='Bronze_Layer/', size=0, modificationTime=1745394195000),
 FileInfo(path='dbfs:/mnt/project2-container/Gold_Layer/', name='Gold_Layer/', size=0, modificationTime=1745524456000),
 FileInfo(path='dbfs:/mnt/project2-container/Silver_Layer/', name='Silver_Layer/', size=0, modificationTime=1745523623000)]

[FileInfo(path='dbfs:/mnt/project2-container/Bronze_Layer/', name='Bronze_Layer/', size=0, modificationTime=1745394195000),
 FileInfo(path='dbfs:/mnt/project2-container/Gold_Layer/', name='Gold_Layer/', size=0, modificationTime=1745524456000),
 FileInfo(path='dbfs:/mnt/project2-container/Silver_Layer/', name='Silver_Layer/', size=0, modificationTime=1745523623000)]

['accounts.csv', 'customers.csv', 'loan_payments.csv', 'loans.csv', 'transactions.csv']


In [0]:
%sql

select * from customers where CustomerId in (80,88)

CustomerId,FirstName,LastName,Address,City,State,Zip,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,Hashkey
80,Emily,Jordan,8888 Airport rd,North Bay,ON,P1B0A1,Gold-Databricks,2025-04-24T19:57:21.635Z,Gold-Databricks-Update,2025-04-24T20:21:44.421Z,4117321498
88,Harjinder,Singh,86 Sunny Ave,Brampton,ON,P0J0A1,Gold-Databricks,2025-04-24T20:21:44.421Z,Gold-Databricks,2025-04-24T20:21:44.421Z,3976200911


In [0]:
%sql

select * from loan_payments@v1

PaymentId,LoanId,PaymentDate,PaymentAmount,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,Hashkey
4,89,2024-01-04,250.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,264335070
23,54,2024-01-23,1200.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,2859663238
92,13,2024-04-01,4650.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,3319289274
6,34,2024-01-06,350.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,3330136997
21,32,2024-01-21,1100.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,2813434115
39,30,2024-02-08,2000.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,2553646516
76,37,2024-03-16,3850.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,175174851
1,45,2024-01-01,100.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,4189237233
45,96,2024-02-14,2300.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,1621318141
34,75,2024-02-03,1750.00,Gold-Databricks,2025-04-24T19:57:24.006Z,Gold-Databricks,2025-04-24T19:57:24.006Z,1366633591


In [0]:
%sql

select * from loans

LoanId,CustomerId,LoanAmount,InterestRate,LoanTerm,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,Hashkey
54,42,30000.50,4.00,48,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,345409227
29,58,32500.25,5.00,36,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,2207440917
72,17,20000.00,3.00,24,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,249839685
88,1,27500.00,3.00,24,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,2680370733
78,4,27500.50,4.00,48,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,1801838765
18,5,27500.50,4.50,48,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,2462199601
70,33,37500.50,4.00,48,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,1845717833
17,99,22500.25,5.50,36,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,4079820486
46,24,17500.50,4.00,48,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,1769370360
8,67,27500.00,3.00,24,Gold-Databricks,2025-04-24T19:57:26.575Z,Gold-Databricks,2025-04-24T19:57:26.575Z,4106075248


In [0]:
%sql

select * from transactions where TransactionId in (100, 101)

TransactionId,AccountId,TransactionDate,TransactionAmount,TransactionType,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,Hashkey
101,50,2024-04-09,400.25,Withdrawal,Gold-Databricks,2025-04-24T20:22:00.655Z,Gold-Databricks,2025-04-24T20:22:00.655Z,605294074
100,50,2024-04-09,550.25,Deposit,Gold-Databricks,2025-04-24T19:57:28.613Z,Gold-Databricks-Update,2025-04-24T20:22:00.655Z,100368312
